In [1]:
##importing the needed libraries for data analysis.
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time

In [2]:
## getting inputs from user for 1) Part name; 2) start and end dates
Part_Name=input("What is the Part Name (e.g. ZM045)?\n")
Start_date=input("Please put in the start date in format mm-dd-yy:\n")
End_date=input("Please put in the end date in format mm-dd-yy:\n")

What is the Part Name (e.g. ZM045)?
ZM045
Please put in the start date in format mm-dd-yy:
03-01-19
Please put in the end date in format mm-dd-yy:
03-13-19


In [3]:
##file path for folder containing the wafer level yield summary reports for all products & all lots
File_Path="//nbpdata01/data2/Test Operations/Data/Conexant/_LotSummaries/"

In [4]:
## list all files under File_Path
folderfiles=os.listdir(File_Path)

In [5]:
## select files only with a specific Part
selected_files=list(filter(lambda x: x.startswith(Part_Name), folderfiles))

In [6]:
##function to grab the last modified time of files.
def get_modification_time(file):
    t=os.path.getmtime(file)
    return time.ctime(t)

In [7]:
# Store the last modified time of selected files in Mod_time
Mod_time=[]

for file in selected_files:
    Mod_time.append(get_modification_time(File_Path+file))


In [8]:
## Create DataFrame with filename and Mod_time, filter to specified date range, store in DataFrame Files_of_interest
File_df=pd.DataFrame({'filename':selected_files,'Mod_time':Mod_time})
File_df['Mod_time']=pd.to_datetime(File_df['Mod_time'])
Files_of_interest=File_df[(File_df['Mod_time']>pd.to_datetime(Start_date)) & (File_df['Mod_time']<pd.to_datetime(End_date)) ]
Files_of_interest.reset_index(drop=True, inplace=True)

In [9]:
#Read the 1st lot yield data into df1
df1=pd.read_table(File_Path+Files_of_interest['filename'][0])

In [10]:
#Loop through the rest of lots in the list, i.e. Files_of_interest, append to df1.
for file in Files_of_interest['filename'][1:]:
    df1=pd.concat((df1, pd.read_table(File_Path+file)))

In [11]:
# strip spaces from column names
df1.columns=df1.columns.str.strip()

In [12]:
df1['Part']=df1['Rev'].apply(lambda x: x.split('_')[0])

In [13]:
#Create output df: Out_df
Out_df=df1[['Part','Lot','Wafer','Yield','Date','Time']]

In [14]:
Out_df['Probe_date']=pd.to_datetime('2019 '+ Out_df['Date'].str[:3]+' '+Out_df['Date'].str[3:]+' '+Out_df['Time'])
Out_df=Out_df[['Part','Lot','Wafer','Yield','Probe_date']]

In [15]:
Out_df.sort_values(by=['Probe_date'], ascending=False, inplace=True)

In [16]:
Out_df.reset_index(drop=True, inplace=True)

In [17]:
#Make dir: C:/Yield_data if not exist
if not os.path.exists('C:/Yield_data'):
    os.makedirs('C:/Yield_data')

In [18]:
#Output Out_df as excel spreadsheet at C:/Yield Data with Part_Name_Start_data(month, day)_End_date(month,day) as file name.
Out_df.to_excel('C:/Yield_data/'+Part_Name+'_'+Start_date[:5]+'_to_'+End_date[:5]+'.xlsx')